In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/intentconan2/validation.csv
/kaggle/input/intentconan2/train.csv
/kaggle/input/intentconan2/test.csv


In [14]:
%pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [15]:
from datasets import load_dataset
dataset = load_dataset("cmu-lti/cobracorpus", split="train")
print(dataset[0])


{'prompt': '- those with mental disabilities lack the moral sense that would restrain them from hurting others\\n- society would be better off if the mentally ill were not allowed to reproduce\\n- it would be easier if the mentally ill were just kept in jail\\n- spending taxpayer dollars on support for mental illness is just a waste of mone\\n- we as a society should not take care of those with mental illness\\n- mentally insane people cannot have a good quality of life\\n-', 'generation_method': 'top-k', 'group': 'mental_dis', 'prompt_label': 1, 'roberta_prediction': 0.001, 'examples': "[Statement] They have been bred to be good at sports and entertainment, but not much else[/]\n[Context of statement] casual conversation between friends[/]\n[Speaker identity/characteristics] white person[/]\n[Listener identity/characteristics] white person[/]\n[Intent] The speaker is trying to convey their general stereotype about a certain group of people who are often sportspersons or entertainers[/

In [ ]:
df = dataset.to_pandas()
selected_columns = ["text", "intent", "targetGroup", "relevantPowerDynamics", "implication"]
df_selected = df[selected_columns]

df_selected.to_csv("cobracorpus.csv", index=False)


In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

df_selected.fillna("None", inplace=True)

def prepare_row(row):
    input_text = f"Hate Speech: {row['text']}"
    output_text = f"Intent: {row['intent']}. Target Group: {row['targetGroup']}. Power Dynamics: {row['relevantPowerDynamics']}. Implication: {row['implication']}."
    return pd.Series([input_text, output_text])

df_selected[["input", "output"]] = df_selected.apply(prepare_row, axis=1)

dataset = Dataset.from_pandas(df_selected[["input", "output"]])

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base")

def tokenize(batch):
    model_inputs = tokenizer(batch["input"], padding="max_length", truncation=True, max_length=256)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(batch["output"], padding="max_length", truncation=True, max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(tokenize, batched=True)



<ipython-input-17-eca112493e35>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected.fillna("None", inplace=True)
<ipython-input-17-eca112493e35>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected[["input", "output"]] = df_selected.apply(prepare_row, axis=1)
<ipython-input-17-eca112493e35>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

Map:   0%|          | 0/31582 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [18]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./bart_pragmatic_model",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    learning_rate=5e-5,
    logging_dir="./logs",
    evaluation_strategy="no",  
    save_strategy="epoch",
    weight_decay=0.01,
    report_to="none"
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)


trainer.train()

trainer.save_model("./bart_pragmatic_model")
tokenizer.save_pretrained("./bart_pragmatic_model")


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-18-67c90cc64445>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss
500,0.826300
1000,0.336500
1500,0.327300
2000,0.313000
2500,0.308600
3000,0.306100
3500,0.301300
4000,0.292000
4500,0.269300
5000,0.265600


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('./bart_pragmatic_model/tokenizer_config.json',
 './bart_pragmatic_model/special_tokens_map.json',
 './bart_pragmatic_model/vocab.json',
 './bart_pragmatic_model/merges.txt',
 './bart_pragmatic_model/added_tokens.json',
 './bart_pragmatic_model/tokenizer.json')

In [19]:
import shutil
shutil.make_archive('/kaggle/working/bart_pragmatic_model', 'zip', '/kaggle/working/bart_pragmatic_model')


'/kaggle/working/bart_pragmatic_model.zip'

## LoRA based finetuning 

In [40]:
import pandas as pd
from datasets import Dataset


train_df = pd.read_csv("/kaggle/input/intentconan2/train.csv")
val_df = pd.read_csv("/kaggle/input/intentconan2/validation.csv")


train_df = train_df.dropna(subset=["hatespeech", "counterspeech", "csType"])
val_df = val_df.dropna(subset=["hatespeech", "counterspeech", "csType"])


train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)


In [41]:
def preprocess_function(examples):
    inputs = [f"Hate Speech: {hs} Intent: {intent}" for hs, intent in zip(examples["hatespeech"], examples["csType"])]
    targets = examples["counterspeech"]

    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=256, truncation=True, padding="max_length")

    # Replace padding token ids with -100
    labels["input_ids"] = [
        [(token if token != tokenizer.pad_token_id else -100) for token in label] for label in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [42]:
from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

tokenized_datasets = DatasetDict({
    "train": train_dataset.map(preprocess_function, batched=True),
    "validation": val_dataset.map(preprocess_function, batched=True),
})


Map:   0%|          | 0/9532 [00:00<?, ? examples/s]

Map:   0%|          | 0/1470 [00:00<?, ? examples/s]

In [44]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

base_model_path = "./bart_pragmatic_model"  # Replace with actual path if different

tokenizer = AutoTokenizer.from_pretrained(base_model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(base_model_path)


In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./bart_intent_finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,  
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,
    fp16=True,  # if GPU with mixed precision
    logging_dir="./logs",
    logging_steps=100,
    report_to="none"
)



from transformers import Seq2SeqTrainer, DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-46-7a36fbb52f4f>:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss
1,2.161600,2.070786
2,1.924100,1.976386
3,1.868100,1.942373
4,1.840100,1.933194


TrainOutput(global_step=4768, training_loss=2.0242917105655542, metrics={'train_runtime': 2386.4573, 'train_samples_per_second': 15.977, 'train_steps_per_second': 1.998, 'total_flos': 1.162401583988736e+16, 'train_loss': 2.0242917105655542, 'epoch': 4.0})

In [48]:
tokenizer.save_pretrained("./bart_intent_finetuned")
model.save_pretrained("./bart_intent_finetuned")


In [50]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

final_model_path = "./bart_intent_finetuned"
tokenizer = AutoTokenizer.from_pretrained(final_model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(final_model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

df_test = pd.read_csv("/kaggle/input/intentconan2/test.csv")


for i in range(10,20):
    hs = df_test.loc[i, "hatespeech"]
    intent = df_test.loc[i, "csType"]
    gt_cs = df_test.loc[i, "counterspeech"]

    
    input_text = f"Hate Speech: {hs} Intent: {intent}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            num_beams=4,
            do_sample=False,
            early_stopping=True,
            eos_token_id=tokenizer.eos_token_id
        )

    pred_cs = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print(f"\n=== Sample {i+1} ===")
    print(f"Hate Speech: {hs}")
    print(f"Intent: {intent}")
    print(f"Ground Truth CS:\n{gt_cs}")
    print(f"Generated CS:\n{pred_cs}")



=== Sample 11 ===
Hate Speech: We began to break down when we allowed women to votemeddle in our civilizations that we had built and maintained for thousands of years. A completely different energy and thought pattern. They are weak and brought our society down to their emotional soft level.
Intent: Questioning
Ground Truth CS:
What is wrong with different thought patterns? isn't our country build on the ideals of unity in diversity?
Generated CS:
Why do you believe that women are weak and incapable of thinking? can you provide evidence to support this claim?

=== Sample 12 ===
Hate Speech: We began to break down when we allowed women to votemeddle in our civilizations that we had built and maintained for thousands of years. A completely different energy and thought pattern. They are weak and brought our society down to their emotional soft level.
Intent: Denouncing
Ground Truth CS:
You seem to have a messed up idea of society. the idea that women are soft and incapable is unfounded a

In [51]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

df_test = pd.read_csv("/kaggle/input/intentconan2/test.csv")

generated_counterspeeches = []

for idx, row in df_test.iterrows():
    hatespeech = row["hatespeech"]
    intent = row["csType"]

    # Prepare input format
    input_text = f"Hate Speech: {hatespeech} Intent: {intent}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=150,
            num_beams=4,
            do_sample=False,
            early_stopping=True,
            eos_token_id=tokenizer.eos_token_id
        )

    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    generated_counterspeeches.append(generated_text)

df_test["generated_counterspeech"] = generated_counterspeeches
df_test.to_csv("bart_cobracorpus_predictions.csv", index=False)

print(" Saved full predictions to 'bart_intent_finetuned_predictions.csv'")


 Saved full predictions to 'bart_intent_finetuned_predictions.csv'


In [56]:
import shutil

# Create a zip file of the directory
shutil.make_archive("bart_intent_finetuned", 'zip', "/kaggle/working/bart_intent_finetuned")


'/kaggle/working/bart_intent_finetuned.zip'